# Week 9 — Toy Causal Discovery (Linear SEM)
We generate data from a small DAG and recover edges using simple regressions.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rng = np.random.default_rng(1)

A = np.array([
    [0,1,0,0],
    [0,0,1,0],
    [0,0,0,1],
    [0,0,0,0],
], dtype=int)
W = A * rng.uniform(0.7, 1.2, size=A.shape)

def topo_order(A):
    d = A.shape[0]
    indeg = A.sum(0).astype(int).tolist()
    order, S = [], [i for i in range(d) if indeg[i]==0]
    while S:
        v = S.pop()
        order.append(v)
        for w in range(d):
            if A[v,w] != 0:
                indeg[w] -= 1
                if indeg[w] == 0:
                    S.append(w)
    return order

def sample_sem(W, n, rng):
    d = W.shape[0]
    order = topo_order((W != 0).astype(int))
    X = np.zeros((n, d), dtype=float)
    for t in range(n):
        for j in order:
            parents = np.where(W[:, j] != 0)[0]
            mean = float(np.dot(X[t, parents], W[parents, j])) if len(parents) else 0.0
            X[t, j] = mean + rng.normal(0, 1.0)
    return X

X = sample_sem(W, 2000, rng)
df = pd.DataFrame(X, columns=['X0','X1','X2','X3'])

def regress_score(df, i, j):
    # score edge i -> j by regression coefficient magnitude
    X = df[[f'X{i}']].values
    y = df[f'X{j}'].values
    beta, *_ = np.linalg.lstsq(X, y, rcond=None)
    return abs(beta[0])

scores = np.zeros_like(A, dtype=float)
for i in range(4):
    for j in range(4):
        if i != j:
            scores[i, j] = regress_score(df, i, j)

thresh = 0.25
A_hat = (scores >= thresh).astype(int)
np.fill_diagonal(A_hat, 0)

print('True adjacency:', '\n', A)
print('Recovered adjacency:', '\n', A_hat)

tp = ((A == 1) & (A_hat == 1)).sum()
fp = ((A == 0) & (A_hat == 1)).sum()
fn = ((A == 1) & (A_hat == 0)).sum()
prec = tp / max(1, tp + fp)
rec = tp / max(1, tp + fn)
print('precision', round(prec, 2), 'recall', round(rec, 2))


In [ ]:
plt.figure(figsize=(5,3))
plt.imshow(scores, cmap='viridis')
plt.colorbar(label='score')
plt.title('Edge scores (regression magnitude)')
plt.xticks(range(4), ['X0','X1','X2','X3'])
plt.yticks(range(4), ['X0','X1','X2','X3'])
plt.tight_layout()
plt.show()
